In [1]:
import warnings
warnings.filterwarnings("ignore")

from DataUtils import *
from hfUtils import *

from tqdm.auto import tqdm

import numpy as np
import torch

2024-11-13 23:52:53.069249: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 23:52:53.075943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731559973.083930 3558848 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731559973.086352 3558848 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 23:52:53.095235: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# First Load The Data to RAM
Below is the function to call to get the data loaded and formatted into prompts.

In [2]:
train_prompts, test_prompts, subjects = load_mmlu_to_ram('./MMLU')

print some examples

In [3]:
n_examples = 2
n_smaples = 1

for i in range(n_examples):
    rnd_subject = np.random.choice(len(subjects))
    
    print('Subject:', subjects[rnd_subject])
    print("--------------------")
    print("Training Prompt:")
    print(train_prompts[rnd_subject])
    print('--------------------')
    
    for j in range(n_smaples):
        rnd_question = np.random.choice(len(test_prompts[rnd_subject]))
        
        print('Test Question:', test_prompts[rnd_subject][rnd_question][0])
        print('Expected Answer:', test_prompts[rnd_subject][rnd_question][1])

Subject: college medicine
--------------------
Training Prompt:
The following are multiple choice questions (with answers) about college medicine.

Glucose is transported into the muscle cell:
A. via protein transporters called GLUT4.
B. only in the presence of insulin.
C. via hexokinase.
D. via monocarbylic acid transporters.
Answer: A

Which of the following is not a true statement?
A. Muscle glycogen is broken down enzymatically to glucose-1-phosphate
B. Elite endurance runners have a high proportion of Type I fibres in their leg muscles
C. Liver glycogen is important in the maintenance of the blood glucose concentration
D. Insulin promotes glucose uptake by all tissues in the body
Answer: D

In a genetic test of a newborn, a rare genetic disorder is found that has X-linked recessive transmission. Which of the following statements is likely true regarding the pedigree of this disorder?
A. All descendants on the maternal side will have the disorder.
B. Females will be approximately t

# Testing A Model
For now let's test google/gemma-2b-it

In [4]:
tokenizer, model = load_model_and_tokenizer("meta-llama/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
messages = [
    {"role": "system", "content": "You are a helpful assitant"},
    {"role": "user", "content": "what is the answer to 2+x+5-x+3x=10"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.01,
    top_p=0.1,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


To solve the equation 2 + x + 5 - x + 3x = 10, we need to combine like terms.

First, let's combine the constants (numbers without variables):
2 + 5 = 7

Now, the equation becomes:
7 + x - x + 3x = 10

Next, let's combine the x terms:
-x + 3x = 2x

Now, the equation becomes:
7 + 2x = 10

To isolate the variable x, we need to get rid of the constant 7 on the left side of the equation. We can do this by subtracting 7 from both sides:
7 - 7 + 2x = 10 - 7
0 + 2x = 3

Now, we can simplify the left side of the equation:
2x = 3

Finally, to solve for x, we need to get x by itself. We can do this by dividing both sides of the equation by 2:
2x / 2 = 3 / 2
x = 3/2
x = 1.5

So, the answer to the equation 2 + x + 5 - x + 3x


In [6]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
chat_history = []
chat_history.append({''})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'What is the capital of France? Paris\nWhat is the capital of England? London\nWhat is the capital of Australia? Canberra\nWhat is the capital of Canada? Ottawa\nWhat is the capital of the United States? Washington D.C.\nWhat is the capital of China? Beijing\nWhat is the capital of Japan? Tokyo\nWhat is the capital of India? New Delhi\nWhat is the capital of Brazil? Brasília\nWhat is the capital of Russia? Moscow\nWhat is the capital of South Africa? Pretoria (administrative capital) and Cape Town (legislative capital) and Bloemfontein (judicial capital)\nWhat is the capital of Egypt? Cairo\nWhat is the capital of South Korea? Seoul\nWhat is the capital of Italy? Rome\nWhat is the capital of Germany? Berlin\nWhat is the capital of Spain? Madrid\nWhat is the capital of Poland? Warsaw\nWhat is the capital of Argentina? Buenos Aires\nWhat is the capital of Mexico? Mexico City\nWhat is'}]

In [5]:
# run all test prompts through the model and get the predictions
# from transformers import pipeline
with torch.no_grad():
    predictions = []
    accuracies = []
    for i,subject in enumerate(subjects):
        train_prompt = train_prompts[i]
        predictions.append([])
        accuracies.append(0)
        for j in tqdm(range(len(test_prompts[i]))):
            input_prompt = train_prompt + test_prompts[i][j][0]
            expected_output = test_prompts[i][j][1]
            
            input_ids = tokenizer.encode(input_prompt, return_tensors='pt').to(model.device)
            logits = model(input_ids=input_ids).logits[0, -1]
            
            probs = (
                torch.nn.functional.softmax(
                    torch.tensor(
                        [
                            logits[tokenizer("A").input_ids[-1]],
                            logits[tokenizer("B").input_ids[-1]],
                            logits[tokenizer("C").input_ids[-1]],
                            logits[tokenizer("D").input_ids[-1]],
                        ]
                    ).float(),
                    dim=0,
                )
                .detach()
                .cpu()
                .numpy()
            )
            pred = {0: "A", 1: "B", 2: "C", 3: "D"}[np.argmax(probs)]
        
            # output = model.generate(input_ids, max_length=input_ids.shape[1]+1, num_return_sequences=1, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
            # prediction = tokenizer.decode(output[0], skip_special_tokens=True)[len(input_prompt):]
            
            predictions[-1].append(pred)
            
            if pred == expected_output:
                accuracies[-1] += 1
                
        accuracies[-1] /= len(test_prompts[i])
        print(f"Subject {subject} accuracy: {accuracies[-1]}")

  0%|          | 0/100 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Subject abstract algebra accuracy: 0.35


  0%|          | 0/135 [00:00<?, ?it/s]

Subject anatomy accuracy: 0.6518518518518519


...

Subject us foreign policy accuracy: 0.87


  0%|          | 0/166 [00:00<?, ?it/s]

Subject virology accuracy: 0.5240963855421686


  0%|          | 0/171 [00:00<?, ?it/s]

Subject world religions accuracy: 0.8128654970760234


In [6]:
print(f"Average accuracy: {np.mean(accuracies)*100:.2f}%")

Average accuracy: 67.31%


In [ ]:
n_examples = 2
n_smaples = 1

for i in range(n_examples):
    rnd_subject = np.random.choice(len(subjects))
    
    print('Subject:', subjects[rnd_subject])
    print("--------------------")
    print("Training Prompt:")
    print(train_prompts[rnd_subject])
    print('--------------------')
    
    for j in range(n_smaples):
        rnd_question = np.random.choice(len(test_prompts[rnd_subject]))
        
        print('Test Question:', test_prompts[rnd_subject][rnd_question][0])
        print('Expected Answer:', test_prompts[rnd_subject][rnd_question][1])
        print('Predicted Answer:', predictions[rnd_subject][rnd_question])